In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/term_paper/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Install all dependecies

# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.9") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{torch.__version__[0:5:2]}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# smpl-x
need_smplx=False
try:
    import smplx
except ModuleNotFoundError:
    need_smplx=True

if need_smplx:
    !pip install smplx
    !git clone https://github.com/vchoutas/smplx
    %cd smplx
    !python setup.py install
    %cd ..

In [ ]:
import smplx
import matplotlib.pyplot as plt
from pytorch3d.vis.plotly_vis import plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from torchvision.io import read_image

from pytorch3d.io import load_obj
from pytorch3d.structures import Meshes, packed_to_list
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras,
    PerspectiveCameras,
    PointLights,
    Materials,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    BlendParams,
    TexturesVertex,
    TexturesUV
)

In [ ]:
### Setup
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.cuda.set_device(device)
else:
    device = torch.device('cpu')

In [ ]:
### Load SMPL-X model as Pytorch3d mesh

# if true, use SMPL object for HUMBI ; otherwise use SMPLX model
HUMBI_SMPL = True

if HUMBI_SMPL:
    # TEX_PATH = drive_path + 'humbi_maps/humbi_body_texture/body_texture_medians/median_subject_1.png'
    TEX_PATH = drive_path + 'generated_maps/generated_images/sample_run_256-progress/87.jpg'
    OBJ_PATH = drive_path + 'smpl_bodies/text_uv_coor_smpl.obj'

    tex_img = read_image(TEX_PATH)
    smplx_tex_img = torch.moveaxis(tex_img, 0, 2).unsqueeze_(0).float() * 1.0/255

    mesh = load_obj(OBJ_PATH, load_textures=False)

    verts_uvs = mesh[2].verts_uvs.unsqueeze_(0)
    faces_uvs = mesh[1].textures_idx.unsqueeze_(0)

    smplx_mesh_verts = mesh[0].unsqueeze_(0)
    smplx_mesh_faces = mesh[1].verts_idx.unsqueeze_(0)
    smplx_texture = TexturesUV(maps=smplx_tex_img, faces_uvs=faces_uvs, verts_uvs=verts_uvs)

else:
    SMPLX_MODEL_PATH = drive_path + 'smpl_bodies'

    smplx_model = smplx.SMPLXLayer(SMPLX_MODEL_PATH, gender='neutral')

    smplx_mesh_verts = smplx_model.forward()['vertices']
    smplx_mesh_faces = torch.Tensor(smplx_model.faces.astype('int')).type(torch.int32).unsqueeze(0)
    smplx_verts_colors = torch.ones_like(smplx_mesh_verts)
    smplx_texture = TexturesVertex(verts_features=smplx_verts_colors)

smplx_mesh = Meshes(smplx_mesh_verts, smplx_mesh_faces, smplx_texture)

In [ ]:
### If HUMBI_SMPL = True, visualize the loaded texture
if HUMBI_SMPL:  
    plt.figure(figsize=(10, 10))
    texture_image = smplx_texture.maps_padded()
    plt.imshow(texture_image.squeeze().cpu().numpy())
    plt.axis('off');

    plt.figure(figsize=(10, 10))
    texturesuv_image_matplotlib(smplx_texture, subsample=None)
    plt.axis("off");

In [ ]:
### Setup pytorch3d mesh renderer

def get_mesh_renderer(img_width=1024, img_height=1024, device=device):

    raster_settings = RasterizationSettings(
        image_size=(img_width, img_height), 
        blur_radius=0.0, 
        faces_per_pixel=1,
        max_faces_per_bin=10000
    )

    blend_params = BlendParams(background_color=(0, 0, 0))
    lights = PointLights(device=device, location=[[2.0, 2.0, 2.0]])

    cameras = PerspectiveCameras().to(device)

    mesh_renderer = MeshRenderer(
        rasterizer=MeshRasterizer(
            cameras=cameras, 
            raster_settings=raster_settings
        ),
        shader=SoftPhongShader(
            device=device, 
            cameras=cameras,
            lights=lights,
            blend_params=blend_params
        )
    )

    return mesh_renderer

In [ ]:
### Render test mesh

img_width, img_height = 1024, 1024
mesh_renderer = get_mesh_renderer(img_width, img_height, device)

smplx_mesh = smplx_mesh.to(device)

materials = Materials(device=device, specular_color=[[0.0, 0.0, 0.0]], shininess=0.0)

dist = 1
azim = 20
elev = 20
R, T = look_at_view_transform(dist=dist, elev=elev, azim=azim)

if HUMBI_SMPL:
    min_y = -0.2 ; max_y = 1.8
else:
    min_y = -1.4 ; max_y = 0.6

cameras = FoVOrthographicCameras(min_y=min_y, max_y=max_y, R=R, T=T).to(device)

img = mesh_renderer(smplx_mesh, cameras=cameras, materials=materials)[0, :, :, 0:3].detach().cpu().numpy()

plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(img)

if HUMBI_SMPL:
    final_render = drive_path + 'generated_maps/generated_images/sample_run_256.png'
    plt.savefig(final_render, bbox_inches='tight')

In [ ]:
### Plotly (interactive) visualization

# Convert TexturesUV to TexturesVertex
def convert_to_textureVertex(textures_uv: TexturesUV, meshes:Meshes) -> TexturesVertex:
    verts_colors_packed = torch.zeros_like(meshes.verts_packed())
    verts_colors_packed[meshes.faces_packed()] = textures_uv.faces_verts_textures_packed().to(device)
    return TexturesVertex( packed_to_list(verts_colors_packed, meshes.num_verts_per_mesh()) ) 

vertex_texture = convert_to_textureVertex(smplx_texture, smplx_mesh)

mesh = Meshes(
    verts=smplx_mesh_verts.to(device),   
    faces=smplx_mesh_faces.to(device),
    textures=vertex_texture
)

# Render figure
fig = plot_scene({
    "Humbi generated texture mapped on SMPL body": {
        "plot": mesh
        }
},)

fig.show()